## Table of contents:
* [Q Learning for Frozen Lake v0](#Qtable)
* [Deep Q Learning for Frozen Lake v0](#DeepQ)
* [Version with ptan](#Ptan)

***
# Q Learning for Frozen Lake v0 <a class="anchor" id="Qtable"></a>

## The Algorithm

- **initialize** Hyperparameters, Env
- **Repeat** for certain number of episodes:
    - **Repeat** for maximum steps in one Episode:
        - do a epsilon greedy step
            - with probability $\epsilon$ choose $a$ as 1 of 4 possible random actions <br>
            otherwise $a=\underset{a'}{\operatorname{argmax}}Q(s,a')$
        - Update Q values according to Bellman Equation:<br>
        $Q(s,a)=Q(s,a)+\mathrm{learning\_rate}\cdot\left(R(s,a)+\gamma\cdot\max\limits_{a'}Q(s',a')-Q(s,a)\right)$

In [ ]:
import numpy as np
import gym
import sys
%matplotlib notebook

env = gym.make("FrozenLake-v0")

action_size = env.action_space.n
state_size = env.observation_space.n

qtable = np.zeros((state_size, action_size))

total_episodes = 20000       # Total episodes
learning_rate = 0.7          # Learning rate
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability
decay_rate = 0.005            # Exponential decay rate for exploration prob

directions = {0:'left', 1:'down', 2:'right', 3:'up'}

rewards = []
for episode in range(total_episodes):
    if (episode % (total_episodes/100)) == 0:
        print('{} %'.format(round(episode/total_episodes*100)), end='\r')
    state = env.reset()
    done = False
    reward_tot = 0

    for step in range(max_steps):
        # Epsilon Greedy action choice
        rand = np.random.rand()
        if rand > epsilon:
            action = np.argmax(qtable[state])
        else:
            action = env.action_space.sample()

        # Do an action and update Qtable afterwards
        state_n, reward, done,_ = env.step(action)

        qtable[state,action] = qtable[state,action] + learning_rate * (reward + gamma*np.max(qtable[state_n]) - qtable[state,action])

        state = state_n

        if done == True:
            reward_tot = reward
            break;

    # make epsilon smaller exponentially and append reward of episode to list of all rewards
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    rewards.append(reward_tot)

print('{} %'.format(100), end='\r')

#np.save('Data/qtable', qtable)
print("\nMean reward: " + str(sum(rewards[-100:])/100))

#qtable = np.load("Data/qtable.npy")

#print(qtable)

## Test Q table

In [ ]:
import sys

def testQTable(output):
    ''' Test Q table and print out average of 100 episodes (if output == False)
    or print one trajectory into file (if output == True)
    '''
    step_count = 0
    if output == True:
        standard_out = sys.stdout
        sys.stdout = open('Render.txt', 'w')

        tries = 1
    else:
        tries = 100
    mean_reward = 0
    for i in range(tries):
        state = env.reset()
        done = False
        tot_reward = 0
        if output == True:
            env.render()
        for step in range(max_steps):
            action = np.argmax(qtable[state])
            state_n, reward, done, info = env.step(action)
            tot_reward += reward
            if output == True:
                env.render()
            state = state_n

            if done == True:
                step_count = step+1
                mean_reward += tot_reward
                break

    mean_reward /= tries
    if output == True:
        sys.stdout = standard_out
        print('Reward: {} after {} steps'.format(mean_reward, step_count))
    else:
        print('Average reward: {}'.format(mean_reward))

    env.reset()
    env.close()

testQTable(output=True)

***
# Deep Q Learning for Frozen Lake v0 <a class="anchor" id="DeepQ"></a>

## The Algorithm

- **initialize** Hyperparameters, Agent/Env, DQN and Target DQN 
- **Transform** Observations to one hot encoding: e.g. **3 -> [0,0,0,1,0,...,0]**
- **Repeat** until mean reward of last 100 games has exceeded reward-bound:
    - do a epsilon greedy step
        - with probability $\epsilon$ choose $a$ as 1 of 4 possible random actions <br>
        otherwise $a=\underset{a'}{\operatorname{argmax}}Q(s,a'|\theta)$
    - put old state $s$, action $a$, reward $r$, done flag, new state $s'$ in experience buffer
    - if episode is done:<br>
        - reset Agent/Env
    - if length of buffer is smaller than certain limit
        - do nothing and repeat loop
    - if current step is multiple of some certain synchronization number
        - updates target nets weights with net weights <br>
        $\hat{\theta}\leftarrow\theta$
    - Sample batch of batch size 100
    - calculate loss of sampled batch btw. net and target net:
        - Loss $L = \left(Q\left(s,a|\theta\right)-R(s,a)+\gamma\cdot\max\limits_{a'}\hat{Q}\left(s',a'|\hat{\theta}\right)\right)^2$ <br>
        or $L = \left(Q\left(s,a|\theta\right)-R(s,a)\right)^2$ if $s'$ is final state
    - backpropagate and update weights

In [ ]:
import gym
import time
import numpy as np
import collections
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

DEFAULT_ENV_NAME = "FrozenLake-v0"
MEAN_REWARD_BOUND = 0.6
HIDDEN_LAYER = 128
GAMMA = 0.95
# Higher batch size to have at least one succsessfull sample for training
BATCH_SIZE = 100
BUFFER_SIZE = 50000
# Lower learning rate to average more samples
LEARNING_RATE = 0.001
SYNC_TARGET_FRAMES = 100
REPLAY_START_SIZE = 2000

EPSILON_STEPS = 5000
EPSILON_START = 1.0
EPSILON_FINAL = 0.02

'''
Defining the Deep NN
'''
class DQN(nn.Module):
    ''' Define Deep Neural Network of form input -> hidden -> output
    '''
    def __init__(self, input_nr, action_nr):
        nn.Module.__init__(self)
        self.layers = nn.Sequential(
            nn.Linear(input_nr, HIDDEN_LAYER),
            nn.ReLU(),
            nn.Linear(HIDDEN_LAYER, action_nr)
        )

    def forward(self, x):
        return self.layers(x.float())

class DiscreteOneHotWrapper(gym.ObservationWrapper):
    ''' Wrap observed state in one hot encoding
    '''
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        self.observation_space = gym.spaces.Box(0.0, 1.0, (env.observation_space.n, ), dtype=np.float32)

    def observation(self, observation):
        ''' get number 1-15  as observation and make it to float array
        '''
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

#env = gym.wrappers.Monitor(DiscreteOneHotWrapper(gym.make(DEFAULT_ENV_NAME)),'TestMonitor')
env = DiscreteOneHotWrapper(gym.make(DEFAULT_ENV_NAME))
Experience = collections.namedtuple('Experience', field_names=['state', 'action', 'reward', 'done', 'new_state'])


class ExperienceBuffer:
    ''' Experience Buffer class with buffer as collections.deque object with length of given capacity
    methods:
        __init__:   initialize the buffer
        __len__:    return the length
        append:     append one given experience and append it on the right
        sample:     sample mini batch from stored experiences and return them as seperated numpy arrays
                    also: delete sampled items (?)
    '''
    # Buffer to store the experiences to learn from
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        # queue one element into the buffer
        self.buffer.append(experience)

    def sample(self, batch_size):
        # sample batch_size elements from the buffer and return the experience
        # as numpy arrays
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
        
        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
               np.array(dones, dtype=np.uint8), np.array(next_states)


class Agent:
    ''' Agent class with environment and experience buffer
    methods:
        __init__:   initialize of environment & buffer
        _reset:     reset of environment and render state if specified (render==True)
        play_step:  make an epsilon greedy step and store it in the experience buffer as specified in text
                    above the code
                    return reward if episode is done else return None
        close:      close the environment
    '''
    def __init__(self, env, exp_buffer, render=False):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset(render)

    #def state_to_one_hot(self):
    #    self.state = F.one_hot(torch.tensor(self.state), num_classes=16)

    def _reset(self, render=False):
        self.state = self.env.reset()
        if render == True:
            self.env.render()

    def play_step(self, net, epsilon=0.0, device="cpu", render=False):
        done_reward = None

        if np.random.random() < epsilon:
            action = self.env.action_space.sample()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a)
            q_vals_v = net(state_v)
            act_v = torch.max(q_vals_v, dim=1)[1]
            action = act_v.item()

        # do step in the environment
        new_state, reward, is_done, _ = self.env.step(action)
        if render == True:
            self.env.render()

        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = reward
            self._reset()
        return done_reward
    
    def close(self):
        self.env.close()

def calc_loss(batch, net, tgt_net, device="cpu"):
    ''' Calculate loss according to description above this code
    '''
    states, actions, rewards, dones, next_states = batch

    states_v = torch.tensor(states)
    next_states_v = torch.tensor(next_states)
    actions_v = torch.tensor(actions)
    rewards_v = torch.tensor(rewards)
    done_mask = torch.ByteTensor(dones)

    state_action_values = net(states_v).gather(1, actions_v.unsqueeze(-1).long()).squeeze(-1) #.long() for windows?
    ##### ALTERNATIVE loop
    #mat = net(states_v)
    #state_action_values = torch.zeros(mat.shape[0])
    #for i in range(len(actions_v)):
    #    state_action_values[i] = mat[i, actions_v[i]]

    next_state_values = tgt_net(next_states_v).max(1)[0]
    next_state_values[done_mask] = 0.0
    next_state_values = next_state_values.detach()

    expected_state_action_values = rewards_v + GAMMA * next_state_values
    return nn.MSELoss()(state_action_values, expected_state_action_values)
######################################################################

#print_flag = True
net = DQN(env.observation_space.shape[0], env.action_space.n)
tgt_net = DQN(env.observation_space.shape[0], env.action_space.n)
net=net.float()
tgt_net=tgt_net.float()
writer = SummaryWriter(comment="-" + DEFAULT_ENV_NAME)
print(net)

buffer = ExperienceBuffer(BUFFER_SIZE)
agent = Agent(env, buffer)
epsilon = EPSILON_START

optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
start_time = ts
best_mean_reward = None


while True:
    frame_idx += 1
    # Epsilon decay linear or exponentially
    epsilon = max(EPSILON_FINAL, EPSILON_START - frame_idx / EPSILON_STEPS)
    #epsilon = EPSILON_FINAL + (EPSILON_START-EPSILON_FINAL)*np.exp(-frame_idx/EPSILON_STEPS)

    done_reward = agent.play_step(net, epsilon, device="gpu")
    if done_reward is not None:
        total_rewards.append(done_reward)
        #speed = 1 #(frame_idx - ts_frame) / (time.time() - ts)
        #ts_frame = frame_idx
        #ts = time.time()
        mean_reward = np.mean(total_rewards[-100:])
        print("%d: done %d games, mean reward %.3f, eps %.2f" % (
            frame_idx, len(total_rewards), mean_reward, epsilon))
        writer.add_scalar("epsilon", epsilon, frame_idx)
        writer.add_scalar("reward_100", mean_reward, frame_idx)
        writer.add_scalar("reward", done_reward, frame_idx)
        #if best_mean_reward is None or best_mean_reward < mean_reward:
        #    torch.save(net.state_dict(), DEFAULT_ENV_NAME + "-best.dat")
        #    if best_mean_reward is not None:
        #        print("Best mean reward updated %.3f -> %.3f, model saved" % (best_mean_reward, mean_reward))
        #    best_mean_reward = mean_reward
        if mean_reward > MEAN_REWARD_BOUND:
            print("Solved in %d frames!" % frame_idx)
            print('Solved in {} seconds'.format(time.time()-start_time))
            break

    if len(buffer) < REPLAY_START_SIZE:
        continue

    if frame_idx % SYNC_TARGET_FRAMES == 0:
        tgt_net.load_state_dict(net.state_dict())
        print("Target net update, {}".format(frame_idx))

    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    loss_t = calc_loss(batch, net, tgt_net, device="gpu")
    loss_t.backward()
    optimizer.step()

writer.close()

## Test Deep Q network

In [ ]:
import sys

def testDeepQ(output):
    ''' Test Deep Q Agent and print out average of 100 episodes (if output == False)
    or print one trajectory into file (if output == True)
    '''
    mean_reward = 0
    step_count = 0

    if output == True:
        standard_out = sys.stdout
        sys.stdout = open('Render.txt', 'w')

        tries = 1
    else:
        tries = 100
    i = 0
    if output == True:
        agent._reset(render=True)
    else:
        agent._reset()
    while i < tries:
        step_count += 1
        if output == True:
            reward = agent.play_step(net, device="gpu", render=True)
        else:
            reward = agent.play_step(net, device="gpu")
        
        if reward is not None:
            mean_reward += reward
            i += 1
    
    mean_reward /= tries
    if output == True:
        sys.stdout = standard_out
        print('Reward: {} after {} steps'.format(mean_reward, step_count))
    else:
        print('Average reward: {}'.format(mean_reward))

    agent.close()

testDeepQ(output=True)

In [ ]:
sys.stdout = standard_out

***
# Version with ptan <a class="anchor" id="Ptan"></a>

In [ ]:
import gym
import ptan
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

MEAN_REWARD_BOUND = 0.6
GAMMA = 0.95
LEARNING_RATE = 0.001
BATCH_SIZE = 100

EPSILON_START = 1.0
EPSILON_STOP = 0.02
EPSILON_STEPS = 5000

REPLAY_BUFFER = 50000


class DQN(nn.Module):
    def __init__(self, input_size, n_actions):
        super(DQN, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
        )

    def forward(self, x):
        return self.net(x)

class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        self.observation_space = gym.spaces.Box(0.0, 1.0, (env.observation_space.n, ), dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

def calc_target(net, local_reward, next_state):
    if next_state is None:
        return local_reward
    state_v = torch.tensor([next_state], dtype=torch.float32)
    next_q_v = net(state_v)
    best_q = next_q_v.max(dim=1)[0].item()
    return local_reward + GAMMA * best_q


if __name__ == "__main__":
    env = DiscreteOneHotWrapper(gym.make("FrozenLake-v0"))
    writer = SummaryWriter(comment="-FrozenLakev0_ptan")
    net = DQN(env.observation_space.shape[0], env.action_space.n)
    print(net)

    selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=EPSILON_START)
    agent = ptan.agent.DQNAgent(net, selector)
    exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=GAMMA, steps_count=1)
    replay_buffer = ptan.experience.ExperienceReplayBuffer(exp_source, REPLAY_BUFFER)

    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    mse_loss = nn.MSELoss()

    total_rewards = []
    step_idx = 0
    done_episodes = 0

    while True:
        step_idx += 1
        selector.epsilon = max(EPSILON_STOP, EPSILON_START - step_idx / EPSILON_STEPS)
        replay_buffer.populate(1)

        if len(replay_buffer) < BATCH_SIZE:
            continue

        # sample batch
        batch = replay_buffer.sample(BATCH_SIZE)
        batch_states = [exp.state for exp in batch]
        batch_actions = [exp.action for exp in batch]
        batch_targets = [calc_target(net, exp.reward, exp.last_state) for exp in batch]
        # train
        optimizer.zero_grad()
        states_v = torch.FloatTensor(batch_states)
        net_q_v = net(states_v)

        target_q = net_q_v.data.numpy().copy()
        target_q[range(BATCH_SIZE), batch_actions] = batch_targets
        target_q_v = torch.tensor(target_q)
        loss_v = mse_loss(net_q_v, target_q_v)
        loss_v.backward()
        optimizer.step()

        # handle new rewards
        new_rewards = exp_source.pop_total_rewards()
        if new_rewards:
            done_episodes += 1
            reward = new_rewards[0]
            total_rewards.append(reward)
            mean_rewards = float(np.mean(total_rewards[-100:]))
            print("%d: reward: %6.2f, mean_100: %6.2f, epsilon: %.2f, episodes: %d" % (
                step_idx, reward, mean_rewards, selector.epsilon, done_episodes))
            writer.add_scalar("reward", reward, step_idx)
            writer.add_scalar("reward_100", mean_rewards, step_idx)
            writer.add_scalar("epsilon", selector.epsilon, step_idx)
            if mean_rewards > MEAN_REWARD_BOUND:
                print("Solved in %d steps and %d episodes!" % (step_idx, done_episodes))
                break
    writer.close()

## Test Version with ptan

In [ ]:
import sys

def testDeepQ_ptan(output):
    ''' Test Deep Q Agent and print out average of 100 episodes (if output == False)
    or print one trajectory into file (if output == True)
    '''
    mean_reward = 0

    if output == True:
        standard_out = sys.stdout
        sys.stdout = open('Render.txt', 'w')

        tries = 1
    else:
        tries = 100

    state = env.reset()
    if output == True:
        env.render()

    i = 0
    steps = 0
    while i < tries:
        state_v = torch.FloatTensor(state)
        action = torch.max(net(state_v), dim=0)[1]
        action_v = action.item()
        state, reward, done, _ = env.step(action_v)
        steps += 1
        if output == True:
            env.render()
        if done == True:
            mean_reward += reward
            i +=1
            state = env.reset()

    mean_reward /= tries
    if output == True:
        sys.stdout = standard_out
        print('Reward: {} after {} steps'.format(mean_reward, steps))
    else:
        print('Average reward: {}'.format(mean_reward))

    env.close()

testDeepQ_ptan(output=False)